In [1]:
!pip install transformers
!pip install datasets

In [2]:
!pip install evaluate


In [3]:
from datasets import load_dataset
#dataset = load_dataset('GonzaloA/fake_news')
dataset = load_dataset('amazon_polarity')

In [4]:
model_checkpoint = 'bert-base-uncased'


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)


In [7]:
from datasets import load_from_disk
# tokenized_datasets_bert = dataset.map(tokenize_function, batched=True)
# tokenized_datasets_bert.save_to_disk("/Users/liang/Downloads/tokenized_datasetbert")
# 加载tokenized数据集
tokenized_datasets_bert = load_from_disk("/Users/liang/Downloads/tokenized_datasetbert")

In [8]:
import os

path = "/Users/liang/Downloads/tokenized_datasetbert"
if os.path.exists(path):
    print("資料集已儲存在指定的路徑。")
else:
    print("資料集尚未儲存。")

資料集已儲存在指定的路徑。


In [9]:
small_train_dataset = tokenized_datasets_bert["train"].shuffle(seed=42).select(range(1500))
small_eval_dataset = tokenized_datasets_bert["test"].shuffle(seed=42).select(range(1500))

In [10]:
!pip install accelerate -U

In [11]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
for param in model.bert.parameters():
    param.requires_grad = False


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [14]:
# from accelerate import Accelerator

In [20]:
from transformers import Trainer, TrainingArguments
# training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",learning_rate=5e-05,per_device_train_batch_size= 1,
#                                   per_device_eval_batch_size=8,seed=42,adam_beta1 = 0.9,adam_beta2=0.999,lr_scheduler_type= 'linear',
#                                  warmup_steps= 1782000,adam_epsilon=1e-8,num_train_epochs=10)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)


In [21]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689577,0.540000


TrainOutput(global_step=188, training_loss=0.7021401587952959, metrics={'train_runtime': 8942.9034, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.021, 'total_flos': 394666583040000.0, 'train_loss': 0.7021401587952959, 'epoch': 1.0})

In [22]:
model.eval()
evaluation_results = trainer.evaluate()
print(evaluation_results)


KeyboardInterrupt: ignored